In [1]:
# http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
import glob
import pathlib
import re


remove_marks_regex = re.compile('[,\.\(\)\[\]\*:;]|<.*?>')
shift_marks_regex = re.compile('([?!])')

# 長い文字列をトークンidのリストに変換する関数
def text2ids(text, vocab_dict):
    text = remove_marks_regex.sub('', text)
    text = shift_marks_regex.sub(r' \1 ', text)
    tokens = text.split()
    return [vocab_dict.get(token, 0) for token in tokens]

# idのリストをLongTensorに変換する関数
def list2tensor(token_idxes, max_len=100, padding=True):
    if len(token_idxes) > max_len:
        token_idxes = token_idxes[:max_len]
    n_tokens = len(token_idxes)
    if padding:
        token_idxes = token_idxes + [0] * (max_len - len(token_idxes))
    return torch.LongTensor(token_idxes), n_tokens

In [2]:
from torch.utils.data import Dataset


# text2ids/list2tensorを使ってDatasetを作る
class IMDBDataset(Dataset):
    # テキストファイルのパスとラベルをまとめたtupleのリストを作る
    def __init__(self, dir_path, train=True, max_len=100, padding=True):
        self.max_len = max_len
        self.padding = padding
        path = pathlib.Path(dir_path)
        vocab_path = path.joinpath('imdb.vocab')
        # ボキャブラリファイルを読み込み、行ごとに分割
        self.vocab_array = vocab_path.open().read().strip().splitlines()
        # 単語をキーとし、値がIDのdictを作る
        self.vocab_dict = dict((w, i+1) for (i, w) in enumerate(self.vocab_array))
        if train:
            target_path = path.joinpath('train')
        else:
            target_path = path.joinpath('test')
        pos_files = sorted(glob.glob(str(target_path.joinpath('pos/*.txt'))))
        neg_files = sorted(glob.glob(str(target_path.joinpath('neg/*.txt'))))
        # posは1、negは0のlabelを付けて、(file_path, label)のtupleのリストを作成
        self.labeled_files = list(zip([0]*len(neg_files), neg_files)) + list(zip([1]*len(pos_files), pos_files))
        
    @property
    def vocab_size(self):
        return len(self.vocab_array)
    
    def __len__(self):
        return len(self.labeled_files)
    
    # ファイルを読み取ってTensorに変換する
    def __getitem__(self, idx):
        label, f = self.labeled_files[idx]
        # ファイルのテキストデータを読み取って小文字に変換
        data = open(f).read().lower()
        # テキストデータをidのリストに変換
        data = text2ids(data, self.vocab_dict)
        data, n = list2tensor(data, self.max_len, self.padding)
        return data, label, n

In [3]:
from torch.utils.data import DataLoader


train_data = IMDBDataset('./aclImdb/')
test_data = IMDBDataset('./aclImdb/', train=False)
# DataLoaderによって(batch_size, step_size)になる
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)

In [4]:
from torch import nn


# 1. 入力Xをembeddingでベクトルの時系列に変換する
# 2. ベクトルをRNNにいれる
# 3. 最後に出力が一次元の線形層に入れて二値問題を解く
class SequenceTaggingNet(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50, num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, x, h0=None, l=None):
        # idをembeddingで多次元のベクトルに変換
        # xは(batch_size, step_size)から(batch_size, step_size, embedding_dim)に変換される
        x = self.emb(x)
        # RNNにはLSTMを採用
        # 初期状態h0とともにRNNにxを渡す
        # pytorchのRNNは複数ステップの入力を受け取り、複数の出力と最後の内部状態を返す
        # xは入力(batch_size, step_size, embedding_dim)から出力(batch_size, step_size, hidden_size)に変換される
        # hは内部状態
        x, h = self.lstm(x, h0)
        # 最後のステップのみ取り出す、(batch_size, step_size, hidden_size)から(batch_size, hidden_size)へ
        if l is not None:
            # 入力のもとの長さがある場合はそれを使用する
            x = x[list(range(len(x))), l-1, :]
        else:
            # なければ単純に最後を使用する
            x = x[:, -1, :]
        # 取り出した最後のステップを線形層にいれて2値分類する
        # (batch_size, hidden_size)から(batch_size, 1)へ
        x = self.linear(x)
        # 余分な次元を削除
        # 2クラス問題を識別できる形に変換する
        # (batch_size, 1)から(batch_size)に変換される
        x = x.squeeze()
        return x

In [5]:
def eval_net(net, data_loader):
    net.eval()
    ys = []
    ypreds = []
    # xは時系列トークンid、yはラベル、lは配列による時系列データの長さ
    for x, y, l in data_loader:
        x = V(x, volatile=True)
        y = V(y, volatile=True)
        y_pred = net(x, l=l)
        y_pred = (y_pred > 0).long()
        ys.append(y.data)
        ypreds.append(y_pred.data)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc

In [6]:
import torch
from torch.autograd import Variable as V
from torch import optim
from statistics import mean
from tqdm import tqdm


# embedding層には0を含めるために+1する
# 2層のlstm
net = SequenceTaggingNet(train_data.vocab_size+1, num_layers=2)
opt = optim.Adam(net.parameters())
# pros/consの二値分類のため損失関数はシグモイドを作用させたクロスエントロピー
loss_f = nn.BCEWithLogitsLoss()
for epoch in range(10):
    losses = []
    net.train()
    for x, y, l in tqdm(train_loader):
        x = V(x)
        y = V(y.float()).float()
        y_pred = net(x, l=l)
        loss = loss_f(y_pred, y)
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.data[0])
    train_acc = eval_net(net, train_loader)
    val_acc = eval_net(net, test_loader)
    print(epoch, mean(losses), train_acc, val_acc, flush=True)

100%|██████████| 782/782 [02:47<00:00,  4.67it/s]


0 0.6673651355154374 0.67408 0.65208


100%|██████████| 782/782 [03:05<00:00,  4.21it/s]


1 0.604958693351587 0.71148 0.68224


100%|██████████| 782/782 [03:05<00:00,  4.21it/s]


...

7 0.17621379246806626 0.96136 0.78904


100%|██████████| 782/782 [03:05<00:00,  4.21it/s]


8 0.13551636398448358 0.96004 0.78356


100%|██████████| 782/782 [03:11<00:00,  4.09it/s]


9 0.10801596829400914 0.9394 0.7538


In [7]:
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression

# BoWを使って文脈を加味しないロジスティック回帰モデルで二値分類してみる
train_X, train_y = load_svmlight_file('./aclImdb/train/labeledBow.feat')
test_X, test_y = load_svmlight_file('./aclImdb/test/labeledBow.feat', n_features=train_X.shape[1])
model = LogisticRegression(C=0.1, max_iter=1000)
model.fit(train_X, train_y)
# 訓練データでは良い結果がでるがテストデータではうまくいかない
model.score(train_X, train_y), model.score(test_X, test_y)

(0.89876, 0.39608)

In [8]:
from torch import nn


class SequenceTaggingNet(nn.Module):
    def __init__(self, num_embeddings, embedding_dim=50, hidden_size=50, num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, x, h0=None, l=None):
        x = self.emb(x)
        # pack_padded_sequenceで時系列の長さ分だけRNNの計算をストップする
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, l, batch_first=True)
        x, h = self.lstm(x, h0)
        # 最後のステップを取り出して線形層に入れる
        if l is not None:
            # 長さ情報がある場合は最後の層の内部状態のベクトルを直接利用できる
            # PackedSequenceが入力されたRNNは、PackedSequenceとともに内部状態を返す
            # PackedSequenceを使用してる場合は線形層への入力は、内部状態かpad_packed_sequenceされた出力をいれる
            # LSTMは通常の内部状態の他にブロックセルの状態もあるので、内部状態のみを利用する
            hidden_state, cell_state = h
            x = hidden_state[-1]
        else:
            x = x[:, -1, :]
        x = self.linear(x).squeeze()
        return x

In [9]:
def eval_net(net, data_loader):
    net.eval()
    ys = []
    ypreds = []
    for x, y, l in data_loader:
        # PackedSequenceを利用するにはデータセットを長い順にソートする必要がある
        l, sort_idx = torch.sort(l, descending=True)
        x = V(x, volatile=True)
        y = V(y, volatile=True)
        y_pred = net(x, l=list(l))
        y_pred = (y_pred > 0).long()
        ys.append(y.data)
        ypreds.append(y_pred.data)
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc

In [10]:
import torch
from torch.autograd import Variable as V
from torch import optim
from statistics import mean
from tqdm import tqdm



net = SequenceTaggingNet(train_data.vocab_size+1, num_layers=2)
opt = optim.Adam(net.parameters())
loss_f = nn.BCEWithLogitsLoss()
for epoch in range(10):
    losses = []
    net.train()
    for x, y, l in tqdm(train_loader):
        # PackedSequenceを利用するにはデータセットを長さ順にソートする必要がある
        l, sort_idx = torch.sort(l, descending=True)
        x = x[sort_idx]
        y = y[sort_idx]
        x = V(x)
        y = V(y.float())
        y_pred = net(x, l=list(l))
        loss = loss_f(y_pred, y)
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.data[0])
    train_acc = eval_net(net, train_loader)
    val_acc = eval_net(net, test_loader)
    print(epoch, mean(losses), train_acc, val_acc, flush=True)

100%|██████████| 782/782 [02:10<00:00,  5.98it/s]


0 0.6603945647282978 0.69896 0.67316


100%|██████████| 782/782 [02:57<00:00,  4.42it/s]


1 0.534956069782262 0.80076 0.73644


100%|██████████| 782/782 [03:05<00:00,  4.22it/s]


...

7 0.13930449300967251 0.95608 0.77756


100%|██████████| 782/782 [02:47<00:00,  4.68it/s]


8 0.11233496549003341 0.96652 0.78244


100%|██████████| 782/782 [03:02<00:00,  4.28it/s]


9 0.09667146474878063 0.97496 0.77328
